In [ ]:
# 데이터 구조
import pandas as pd
# 데이터 시각화
import matplotlib.pyplot as plt
import matplotlib
# 데이터 분할: train, test
from sklearn.model_selection import train_test_split
# 회귀 Random Forest
from sklearn.ensemble import RandomForestRegressor
# 최적 모델, 파라미터 탐색
from sklearn.model_selection import GridSearchCV
# 나무구조 시각화
from sklearn.tree import plot_tree

In [ ]:
# 한글 맑은 고딕 적용
matplotlib.rc("font", family = "Malgun Gothic")
# 음수값 표시
matplotlib.rc("axes", unicode_minus = False)

In [ ]:
# 파일명, 변수, 값 등에 한글 포함시 encoding=“euc-kr" 지정
df_raw = pd.read_csv("../data/body.csv")
df_raw.head()
# 목표변수, 설명변수 분리
df_raw_y = df_raw["FAT"]
df_raw_x = df_raw.drop("FAT", axis = 1, inplace = False)
# 학습용/평가용 데이터 분리
df_train_x, df_test_x, df_train_y, df_test_y = train_test_split(df_raw_x, df_raw_y,
test_size = 0.3, random_state = 1234)
print("학습용 데이터의 X 크기 : {}".format(df_train_x.shape))
print("학습용 데이터의 Y 크기 : {}".format(df_train_y.shape))
print("평가용 데이터의 X 크기 : {}".format(df_test_x.shape))
print("평가용 데이터의 Y 크기 : {}".format(df_test_y.shape))

In [ ]:
rf_uncustomized = RandomForestRegressor(random_state=1234)
rf_uncustomized.fit(df_train_x, df_train_y)
# 학습용 데이터 설명력
print("학습용 데이터의 설명력 : {:.3f}".format(rf_uncustomized.score(df_train_x, df_train_y)))
# 평가용 데이터 설명력
print("평가용 데이터의 설명력 : {:.3f}".format(rf_uncustomized.score(df_test_x, df_test_y)))

In [ ]:
estimator = RandomForestRegressor(random_state=1234)
# 구하고자 하는 parameter와 범위
param_grid = {"n_estimators": [100, 150, 200],"max_depth": [5, 6, 7, 8, 9], "min_samples_split": [1, 2, 3, 4],
"min_samples_leaf": [2, 4, 6, 8]}
# 설명력이 높은 최적 parameter 찾기
grid_dt = GridSearchCV(estimator, param_grid, scoring="r2", n_jobs = -1)
grid_dt.fit(df_train_x, df_train_y)
print("best estimator model: \n{}".format(grid_dt.best_estimator_))
print("\nbest parameter: \n{}".format(grid_dt.best_params_))
print("\nbest score: \n{}".format(grid_dt.best_score_.round(3)))

In [ ]:
rf_final = RandomForestRegressor(n_estimators = 100, max_depth = 8, min_samples_split = 2,
min_samples_leaf = 4, random_state=1234)
rf_final.fit(df_train_x, df_train_y)
# 학습용 데이터 설명력
print("학습용 데이터의 설명력: {:.3f}".format(rf_final.score(df_train_x, df_train_y)))
# 평가용 데이터 설명력
330
print("평가용 데이터의 설명력: {:.3f}".format(rf_final.score(df_test_x, df_test_y)))

In [ ]:
# 변수 저장
v_feature_name = df_train_x.columns
# 0번 트리
plt.figure(figsize = (10, 7))
plot_tree(rf_final.estimators_[0], feature_names = v_feature_name, filled = True);
# 1번 트리
plt.figure(figsize = (10, 7))
plot_tree(rf_final.estimators_[1], feature_names = v_feature_name, filled = True);
# 10번 트리
plt.figure(figsize = (10, 7))
plot_tree(rf_final.estimators_[10], feature_names = v_feature_name, filled = True);

In [ ]:
# tree.feature_importances_로 설명변수 중요도 확인 및 테이블로 저장
df_importance = pd.DataFrame()
df_importance["Feature"] = df_train_x.columns
df_importance["Importance"] = rf_final.feature_importances_
# df_feature_importance의 테이블을 중요도 순으로 정렬
df_importance.sort_values("Importance", ascending=False, inplace = True)
df_importance.round(3)